Notebook purpose

- Replicate analysis of davenport2020spending

In [1]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
d = aws.BucketManager('3di-data-mdb/clean')
d.list()

['3di-data-mdb/clean/',
 '3di-data-mdb/clean/data_XX7.parquet',
 '3di-data-mdb/clean/mdb_000.parquet',
 '3di-data-mdb/clean/mdb_777.parquet',
 '3di-data-mdb/clean/mdb_X77.parquet',
 '3di-data-mdb/clean/mdb_costa.parquet']

# Prepare data

In [6]:
def read_sample(sample):
    """Read clean data for sample."""
    fp = f's3://3di-data-mdb/clean/mdb_{SAMPLE}.parquet'
    return aws.s3read_parquet(fp)

def clean_data(df):
    """Prepare data for analysis"""
    return df


SAMPLE = '777'

df = (
    read_sample(SAMPLE)
    .pipe(clean_data)
)

print(df.shape)
df.head(2)

(682656, 28)


,id,date,user_id,amount,desc,merchant,tag_up,user_female,user_lsoa,user_msoa,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,credit_debit,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_recorded_balance,merchant_business_line,tag_auto,tag_manual,updated_flag
0,688261,2012-01-03,777,-400.00,<mdbremoved> - s/o,no merchant,other account,False,e01012553,e02002603,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,debit,2014-07-18,2017-11-13,True,364.22,non merchant mbl,transfers,other account,u
1,688264,2012-01-03,777,-10.27,"9572 30dec11 , mcdonalds , restaurant , winwick road gb - pos",mcdonalds,dining and drinking,False,e01012553,e02002603,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,debit,2014-07-18,2015-03-19,True,364.22,mcdonalds,dining and drinking,no tag,u


In [11]:
df.groupby('user_id').user_id.count().describe()

count      283.000000
mean      2412.212014
std       2341.533982
min          5.000000
25%        698.000000
50%       1610.000000
75%       3463.000000
max      13176.000000
Name: user_id, dtype: float64

In [7]:
df.user_id.nunique()

283

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682656 entries, 0 to 682655
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id                                682656 non-null  int32         
 1   date                              682656 non-null  datetime64[ns]
 2   user_id                           682656 non-null  int32         
 3   amount                            682656 non-null  float32       
 4   desc                              682442 non-null  object        
 5   merchant                          682656 non-null  object        
 6   tag_up                            682656 non-null  object        
 7   user_female                       682656 non-null  bool          
 8   user_lsoa                         678394 non-null  object        
 9   user_msoa                         678394 non-null  object        
 10  user_postcode                   

# Exploration

In [36]:
df.date.describe()

/var/folders/xg/n9p73cf50s52twlnz7z778vr0000gn/T/ipykernel_2430/1371252037.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.date.describe()


count                  682656
unique                   3002
top       2018-04-03 00:00:00
freq                     1940
first     2012-01-03 00:00:00
last      2020-08-16 00:00:00
Name: date, dtype: object

In [28]:
df.tag_up.value_counts()[:5]

no tag                        188717
food, groceries, household     61572
transfers                      44864
cash                           29073
enjoyment                      27999
Name: tag_up, dtype: int64

In [35]:
mask = ((df.tag_up == 'transfers')
        & (df.account_type == 'savings')
        & (df.amount > 0))

df[mask].sort_values(['user_id', 'date'])[['user_id', 'date', 'amount', 'desc']][:100]

,user_id,date,amount,desc
6108,777,2019-02-11,120.000000,from a/c xxxx5781 via mobile - xfer - dpc
6109,777,2019-02-11,60.000000,from a/c xxxx5781 via mobile - xfer - dpc
6141,777,2019-02-25,250.000000,from a/c xxxx5781 via mobile - xfer - dpc
6142,777,2019-02-25,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
6156,777,2019-02-27,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
6252,777,2019-03-29,155.000000,from a/c xxxx5781 via mobile - xfer - dpc
6253,777,2019-03-29,175.000000,from a/c xxxx5781 via mobile - xfer - dpc
6458,777,2019-06-03,600.000000,<mdbremoved>
6504,777,2019-06-18,480.000000,<mdbremoved>
6607,777,2019-07-25,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
